# create observation model


poiché il Pombp ha bisogno di un modello probabilistico per le osservazioni, nello scenario reale è necessario costruirlo.

L'idea è uella di allenare una rete a mappare le osservazioni in un maze deformato in osservazioni del maze originale. in uesto modo




In [1]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque
import itertools
from tqdm import tqdm, trange

import os 
import sys

sys.path.append(os.path.abspath('..'))

from environment.env import MDPGYMGridEnvDeform, create_maze
import matplotlib.pyplot as plt



In [2]:
# maze size
N = 2

# thetas deformations (range(a,b),range(c,d))
l0 = 1
h0 = 5
l1 = 1
h1 = 5

maze = np.load("../maze/maze_2.npy")
env = MDPGYMGridEnvDeform(maze,l0,h0,l1,h1)

env.set_rendering()
env.render()

states = [((x,y,phi),(i,j)) for x in range(1,env.max_shape[0]-1) for y in range(1,env.max_shape[1]-1) for phi in range(4) for i in range(l0,h0) for j in range(l1,h1)] 
actions = [0,1,2,3]
obs = list(itertools.product([0,1], repeat=5))
thetas = [(i,j) for i in range(l0,h0) for j in range(l1,h1)]

state_dict = {state: i for i, state in enumerate(states)}
obs_dict = {obs : i for i, obs in enumerate(obs)}

# Actions are: 0-listen, 1-open-left, 2-open-right
lenS = len(states)
lenA = len(actions)
lenO = len(obs)

print(f"States: {lenS}, Actions: {lenA}, Observations {lenO}, Thetas {thetas}\n")



States: 9216, Actions: 4, Observations 32, Thetas [(1, 1), (1, 2), (1, 3), (1, 4), (2, 1), (2, 2), (2, 3), (2, 4), (3, 1), (3, 2), (3, 3), (3, 4), (4, 1), (4, 2), (4, 3), (4, 4)]



In [ ]:
env.close()

In [41]:
env.render()

In [7]:
print(env.reset(state=(1,1)))
print(env.goal_pos)
env.render()

(OrderedDict([('x', tensor([4], dtype=torch.int32)), ('y', tensor([8], dtype=torch.int32)), ('phi', tensor([0], dtype=torch.int32)), ('theta', tensor([1, 1]))]), {})
[3 3]


In [8]:
def s_to_array(s):
    return [s['x'].item(), s['y'].item(), s['phi'].item(),s['theta'][0].item(), s['theta'][1].item()]

def create_dataset(env: MDPGYMGridEnvDeform, size: int = 100):
    dataset = []
    for _ in tqdm(range(size)):
        s, _  = env.reset(state=(1,1))
        obs = env.get_observation()
        s = s_to_array(s)
        # discard any position that is not in the maze
        if s[0] >= env.original_maze.shape[0] or s[1] >= env.original_maze.shape[1]:
            continue
        for theta in env.deformations:
            # come rimappare la posizione in base alla nuova theta?
            s_new = s[:2] * np.array(theta) 
            s_new = np.append(s_new, s[2])
            s_new = (tuple(s_new), theta)
            # print(s_new)
            obs_new = env.get_observation(s_new)

            dataset.append({
                'obs': obs,
                'obs_new': obs_new,
                'theta': theta,
                's': s,
            })

    return dataset

dataset = create_dataset(env, 50000)

100%|██████████| 50000/50000 [00:09<00:00, 5286.81it/s]


In [9]:
len(dataset), dataset[50]

(21680,
 {'obs': array([1, 0, 0, 1, 1]),
  'obs_new': array([1, 1, 0, 1, 1]),
  'theta': (1, 3),
  's': [2, 1, 2, 1, 1]})